# Data Type I

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset, Dataset
from scipy.stats import norm
from sklearn.metrics import mean_absolute_error, mean_squared_error

## IB-GRU

In [2]:
# Step 1 : Encoder
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, z_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim + hidden_dim, 16),
            nn.ReLU(),
            nn.Linear(16, z_dim * 2)  
        )

    def forward(self, x_t, h_prev):
        inp = torch.cat([x_t, h_prev], dim=-1)
        stats = self.fc(inp)
        mu, logvar = stats.chunk(2, dim=-1)
        return mu, logvar

# Step 2 : Prior
class Prior(nn.Module):
    def __init__(self, hidden_dim, z_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 16),
            nn.ReLU(),
            nn.Linear(16, z_dim * 2)
        )

    def forward(self, h_prev):
        stats = self.fc(h_prev)
        mu, logvar = stats.chunk(2, dim=-1)
        return mu, logvar

# Step 3 : Decoder
class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(z_dim + hidden_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 2)  
        )

    def forward(self, z_t, h_prev):
        inp = torch.cat([z_t, h_prev], dim=-1)
        stats = self.fc(inp)
        mu, logvar = stats.chunk(2, dim=-1)
        return mu, logvar

# Reparameterization trick
def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return mu + eps * std

# Step 4 : RIB_GRU_Model
class RIB_GRU_Model(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, z_dim=16):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.z_dim = z_dim

        self.encoder = Encoder(input_dim, hidden_dim, z_dim)
        self.prior = Prior(hidden_dim, z_dim)
        self.decoder = Decoder(z_dim, hidden_dim)

        self.gru_cell = nn.GRUCell(z_dim, hidden_dim)

    def forward(self, x_t, h_prev):
        # Encoder
        mu_enc, logvar_enc = self.encoder(x_t, h_prev)
        z_t = reparameterize(mu_enc, logvar_enc)

        # Prior
        mu_prior, logvar_prior = self.prior(h_prev)

        # Decoder
        mu_dec, logvar_dec = self.decoder(z_t, h_prev)

        # GRU update
        h_t = self.gru_cell(z_t, h_prev)

        return {
            'z_t': z_t,
            'mu_enc': mu_enc, 'logvar_enc': logvar_enc,
            'mu_prior': mu_prior, 'logvar_prior': logvar_prior,
            'mu_dec': mu_dec, 'logvar_dec': logvar_dec,
            'h_t': h_t
        }
    
# Loss function
LOG_2PI = torch.tensor(np.log(2.0 * np.pi))

def rib_loss(outputs, y_t, beta=0.005):
    
    mu_enc = outputs['mu_enc']
    logvar_enc = outputs['logvar_enc']
    mu_prior = outputs['mu_prior']
    logvar_prior = outputs['logvar_prior']
    mu_dec = outputs['mu_dec']
    logvar_dec = outputs['logvar_dec']

    log_2pi = LOG_2PI.to(y_t.device) 

    # Decoder Gaussian NLL
    var_dec = logvar_dec.exp()
    nll = 0.5 * (log_2pi + logvar_dec + ((y_t - mu_dec) ** 2) / var_dec)
    recon_loss = nll.mean()

    # KL(q || p)
    var_enc = logvar_enc.exp()
    var_prior = logvar_prior.exp()
    kl = 0.5 * torch.sum(logvar_prior - logvar_enc + (var_enc + (mu_enc - mu_prior) ** 2) / var_prior - 1, dim=-1).mean()

    return recon_loss + beta * kl

In [ ]:
# Load and preprocess data
df = pd.read_csv('data/Sunspots_clean.csv')
values = df['Sunspots'].values

# Normalize data to [0, 1]
scaler = MinMaxScaler()
scaled_series = scaler.fit_transform(values.reshape(-1, 1)).flatten()

# Split into training and test sets
train_set = scaled_series[:1000]
test_set = scaled_series[1000:2000]

print (f"Train size: {len(train_set)}, Test size: {len(test_set)}")

# Ensure the sequences are of the same length
X_train = torch.tensor(train_set[:-1]).unsqueeze(0).float()
Y_train = torch.tensor(train_set[1:]).unsqueeze(0).float()

X_val = torch.tensor(test_set[:-1]).unsqueeze(0).float()
Y_val = torch.tensor(test_set[1:]).unsqueeze(0).float()

# Create DataLoader for training and validation sets
train_loader = DataLoader(TensorDataset(X_train, Y_train), batch_size=1, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, Y_val), batch_size=1, shuffle=False)

# Initialize model
rib_gru_model = RIB_GRU_Model(input_dim=1, hidden_dim=32, z_dim=16)
optimizer = optim.Adam(rib_gru_model.parameters(), lr=0.001)
beta = 0.005
num_epochs = 1000

for epoch in range(num_epochs):
    rib_gru_model.train()
    total_loss = 0.0

    for X_batch, Y_batch in train_loader:
        h_t = torch.zeros(X_batch.size(0), rib_gru_model.hidden_dim)

        loss = 0.0
        for t in range(X_batch.size(1)):
            x_t = X_batch[:, t].unsqueeze(-1)
            y_t = Y_batch[:, t].unsqueeze(-1)

            out = rib_gru_model(x_t, h_t)
            h_t = out['h_t']

            # Compute loss
            loss += rib_loss(out, y_t, beta=beta)
            
        # Average loss over the batch
        loss /= X_batch.size(1)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    if (epoch + 1) % 100 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_loss:.4f}")

# 5mins

Train size: 1000, Test size: 1000
Epoch 1/1000 | Train Loss: 1.0913
Epoch 100/1000 | Train Loss: -0.4132
Epoch 200/1000 | Train Loss: -0.5210
Epoch 300/1000 | Train Loss: -1.3404
Epoch 400/1000 | Train Loss: -1.4463
Epoch 500/1000 | Train Loss: -1.4774
Epoch 600/1000 | Train Loss: -1.5172
Epoch 700/1000 | Train Loss: -1.5301
Epoch 800/1000 | Train Loss: -1.4996
Epoch 900/1000 | Train Loss: -1.5180
Epoch 1000/1000 | Train Loss: -1.5305


In [ ]:
# Store metrics from all runs
mae_list = []
rmse_list = []

for run in range(10):

    # Validation
    rib_gru_model.eval()
    preds = []
    trues = []
    vars_all = []

    with torch.no_grad():
        for X_batch, Y_batch in val_loader:

            h_t = torch.zeros(X_batch.size(0), rib_gru_model.hidden_dim)
            pred_mu = []
            pred_var = []

            for t in range(X_batch.size(1)):

                x_t = X_batch[:, t].unsqueeze(-1)
                y_t = Y_batch[:, t].unsqueeze(-1)

                out = rib_gru_model(x_t, h_t)
                h_t = out['h_t']

                mu_dec = out['mu_dec']
                logvar_dec = out['logvar_dec']

                pred_mu.append(mu_dec.squeeze(-1))
                pred_var.append(torch.exp(logvar_dec).squeeze(-1))

            pred_mu = torch.stack(pred_mu, dim=1) 
            pred_var = torch.stack(pred_var, dim=1)  

            preds.append(pred_mu)
            vars_all.append(pred_var)
            trues.append(Y_batch)

    preds = torch.cat(preds, dim=0).squeeze(0).numpy()
    trues = torch.cat(trues, dim=0).squeeze(0).numpy()
    vars_all = torch.cat(vars_all, dim=0).squeeze(0).numpy()

    # Calculate MAE, RMSE
    rib_gru_mae = np.mean(np.abs(preds - trues))
    rib_gru_rmse = np.sqrt(np.mean((preds - trues)**2))

    mae_list.append(rib_gru_mae)
    rmse_list.append(rib_gru_rmse)

    print(f"Run {run+1}: RIB-GRU MAE: {rib_gru_mae:.4f} | RMSE: {rib_gru_rmse:.4f}")

# Compute mean metrics
mean_rib_gru_mae = np.mean(mae_list)
mean_rib_gru_rmse = np.mean(rmse_list)

print("======= FINAL RESULTS =======")
print(f"Mean RIB-GRU MAE: {mean_rib_gru_mae:.4f} | RMSE: {mean_rib_gru_rmse:.4f}")

Run 1: RIB-GRU MAE: 0.0462 | RMSE: 0.0643
Run 2: RIB-GRU MAE: 0.0468 | RMSE: 0.0651
Run 3: RIB-GRU MAE: 0.0467 | RMSE: 0.0656
Run 4: RIB-GRU MAE: 0.0471 | RMSE: 0.0666
Run 5: RIB-GRU MAE: 0.0478 | RMSE: 0.0661
Run 6: RIB-GRU MAE: 0.0466 | RMSE: 0.0654
Run 7: RIB-GRU MAE: 0.0468 | RMSE: 0.0650
Run 8: RIB-GRU MAE: 0.0462 | RMSE: 0.0649
Run 9: RIB-GRU MAE: 0.0471 | RMSE: 0.0657
Run 10: RIB-GRU MAE: 0.0473 | RMSE: 0.0665
======= FINAL RESULTS =======
Mean RIB-GRU MAE: 0.0469 | RMSE: 0.0655


## IProjLSTM

In [3]:
import torch
import torch.nn as nn

# IProjLSTMCell helper networks 
class RefNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(RefNetwork, self).__init__()
        self.W_mu = nn.Linear(input_dim, hidden_dim, bias=False)
        self.U_mu = nn.Linear(hidden_dim, hidden_dim)
        self.W_sigma = nn.Linear(input_dim, hidden_dim, bias=False)
        self.U_sigma = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x_t, h_prev):
        mu_t = self.W_mu(x_t) + self.U_mu(h_prev) 
        log_sigma2_t = self.W_sigma(x_t) + self.U_sigma(h_prev) 
        sigma2_t = torch.exp(log_sigma2_t) 
        return mu_t, sigma2_t

# Constraint for I-projection
class TargetConstraintNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, learn_sigma=True):
        super(TargetConstraintNetwork, self).__init__()
        self.W_target = nn.Linear(input_dim, hidden_dim, bias=False)
        self.U_target = nn.Linear(hidden_dim, hidden_dim)
        self.learn_sigma = learn_sigma
        if learn_sigma: 
            self.W_sigma_target = nn.Linear(input_dim, hidden_dim, bias=False)
            self.U_sigma_target = nn.Linear(hidden_dim, hidden_dim)
        else:
            self.register_buffer('fixed_sigma2', torch.ones(hidden_dim)) 

    def forward(self, x_t, h_prev):
        mu_t_prime = self.W_target(x_t) + self.U_target(h_prev)
        if self.learn_sigma: 
            log_sigma2_prime = self.W_sigma_target(x_t) + self.U_sigma_target(h_prev)
            sigma2_t_prime = torch.exp(log_sigma2_prime)
        else:
            sigma2_t_prime = self.fixed_sigma2
        return mu_t_prime, sigma2_t_prime

# Main IProjLSTM Cell 
class IProjLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.ref_net = RefNetwork(input_dim, hidden_dim)
        self.target_net = TargetConstraintNetwork(input_dim, hidden_dim)
        self.W_o = nn.Linear(input_dim, hidden_dim)
        self.U_o = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x_t, h_prev):
        mu_ref, sigma2_ref = self.ref_net(x_t, h_prev) 
        mu_target, sigma2_target = self.target_net(x_t, h_prev)
        mu_proj, sigma2_proj = mu_target, sigma2_ref  # I-projection

        # Deterministic cell state
        C_t = mu_proj
        o_t = torch.sigmoid(self.W_o(x_t) + self.U_o(h_prev))
        h_t = o_t * torch.tanh(C_t)
        return h_t, C_t

# Full model using the custom cell 
class IProjLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_len):
        super().__init__()
        self.seq_len = seq_len
        self.hidden_dim = hidden_dim
        self.cell = IProjLSTMCell(input_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, 1)

    def forward(self, x_seq):
        batch_size = x_seq.size(0)
        h_t = torch.zeros(batch_size, self.hidden_dim, device=x_seq.device)
        for t in range(self.seq_len):
            x_t = x_seq[:, t, :]
            h_t, C_t = self.cell(x_t, h_t)[:2]
        y_pred = self.output_layer(h_t)
        return y_pred

In [3]:
# Load and preprocess data
df = pd.read_csv('data/Sunspots_clean.csv')  
values = df['Sunspots'].values

# Normalize data to [0, 1]
scaler = MinMaxScaler()
scaled_series = scaler.fit_transform(values.reshape(-1, 1)).flatten()

# Split into training and test sets
train_set = scaled_series[:1000]
test_set = scaled_series[1000:2000]
print(f"Train size: {len(train_set)}, Test size: {len(test_set)}")

# Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len=20): 
        self.series = torch.tensor(series, dtype=torch.float32)
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        x = self.series[idx:idx+self.seq_len]
        y = self.series[idx+self.seq_len]
        return x.unsqueeze(-1), y.unsqueeze(-1)
    
# Parameters
seq_len = 16
batch_size = 32

# DataLoaders
train_dataset = TimeSeriesDataset(train_set, seq_len)
test_dataset = TimeSeriesDataset(test_set, seq_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Train size: 1000, Test size: 1000


In [7]:
# Lists to store metrics for all runs
mae_list = []
rmse_list = []

for run in range(10):
    print(f"\n=== Run {run+1}/10 ===")

    # Model setup
    iproj_model = IProjLSTMModel(input_dim=1, hidden_dim=32, seq_len=seq_len)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    iproj_model = iproj_model.to(device)

    optimizer = torch.optim.Adam(iproj_model.parameters(), lr=0.001)
    criterion = torch.nn.MSELoss()

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        iproj_model.train()
        epoch_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = iproj_model(x_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}")

    # Evaluation
    iproj_model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(device)
            y_pred = iproj_model(x_batch)
            preds.append(y_pred.cpu())
            targets.append(y_batch.cpu())

    preds = torch.cat(preds).squeeze().numpy()
    targets = torch.cat(targets).squeeze().numpy()

    iproj_mae = mean_absolute_error(targets, preds)
    iproj_rmse = np.sqrt(mean_squared_error(targets, preds))

    mae_list.append(iproj_mae)
    rmse_list.append(iproj_rmse)

    print(f"Run {run+1} - Test MAE: {iproj_mae:.4f} | Test RMSE: {iproj_rmse:.4f}")

# Compute mean metrics
mean_iproj_mae = np.mean(mae_list)
mean_iproj_rmse = np.mean(rmse_list)

print("\n======= FINAL RESULTS =======")
print(f"Mean MAE  : {mean_iproj_mae:.4f}")
print(f"Mean RMSE : {mean_iproj_rmse:.4f}")



=== Run 1/10 ===
Epoch 1/100, Train Loss: 0.0086
Epoch 10/100, Train Loss: 0.0037
Epoch 20/100, Train Loss: 0.0037
Epoch 30/100, Train Loss: 0.0038
Epoch 40/100, Train Loss: 0.0037
Epoch 50/100, Train Loss: 0.0037
Epoch 60/100, Train Loss: 0.0037
Epoch 70/100, Train Loss: 0.0037
Epoch 80/100, Train Loss: 0.0037
Epoch 90/100, Train Loss: 0.0037
Epoch 100/100, Train Loss: 0.0037
Run 1 - Test MAE: 0.0466 | Test RMSE: 0.0657

=== Run 2/10 ===
Epoch 1/100, Train Loss: 0.0254
Epoch 10/100, Train Loss: 0.0039
Epoch 20/100, Train Loss: 0.0037
Epoch 30/100, Train Loss: 0.0038
Epoch 40/100, Train Loss: 0.0037
Epoch 50/100, Train Loss: 0.0038
Epoch 60/100, Train Loss: 0.0038
Epoch 70/100, Train Loss: 0.0037
Epoch 80/100, Train Loss: 0.0037
Epoch 90/100, Train Loss: 0.0037
Epoch 100/100, Train Loss: 0.0037
Run 2 - Test MAE: 0.0469 | Test RMSE: 0.0656

=== Run 3/10 ===
Epoch 1/100, Train Loss: 0.0160
Epoch 10/100, Train Loss: 0.0038
Epoch 20/100, Train Loss: 0.0037
Epoch 30/100, Train Loss: 0.0038

## LSTM

In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1, seq_len=20):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, 1)
        self.seq_len = seq_len
        self.hidden_dim = hidden_dim

    def forward(self, x_seq):
        out, _ = self.lstm(x_seq)
        last_hidden = out[:, -1, :]  # take output at final timestep
        y_pred = self.output_layer(last_hidden)
        return y_pred

In [ ]:
# Load and preprocess data
df = pd.read_csv('data/Sunspots_clean.csv')  
values = df['Sunspots'].values

# Normalize data to [0, 1]
scaler = MinMaxScaler()
scaled_series = scaler.fit_transform(values.reshape(-1, 1)).flatten()

# Split into training and test sets
train_set = scaled_series[:1000]
test_set = scaled_series[1000:2000]

print (f"Train size: {len(train_set)}, Test size: {len(test_set)}")

# Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len=20): 
        self.series = torch.tensor(series, dtype=torch.float32)
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        x = self.series[idx:idx+self.seq_len]
        y = self.series[idx+self.seq_len]
        return x.unsqueeze(-1), y.unsqueeze(-1)

# Parameters
seq_len = 16
batch_size = 32

# DataLoaders
train_dataset = TimeSeriesDataset(train_set, seq_len)
test_dataset = TimeSeriesDataset(test_set, seq_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Lists to store metrics for all runs
mae_list = []
rmse_list = []

for run in range(10):
    print(f"\n=== Run {run+1}/10 ===")

    # Initialize model
    lstm_model = LSTMModel(input_dim=1, hidden_dim=32, seq_len=seq_len)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    lstm_model = lstm_model.to(device)

    optimizer_lstm = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    num_epochs = 100
    # Training loop
    for epoch in range(num_epochs):
        lstm_model.train()
        epoch_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer_lstm.zero_grad()
            y_pred = lstm_model(x_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer_lstm.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)

        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"LSTM Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}")

    # Evaluation
    lstm_model.eval()
    lstm_preds = []
    lstm_targets = []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(device)
            y_pred = lstm_model(x_batch)
            lstm_preds.append(y_pred.cpu())
            lstm_targets.append(y_batch.cpu())

    lstm_preds = torch.cat(lstm_preds).squeeze().numpy()
    lstm_targets = torch.cat(lstm_targets).squeeze().numpy()

    lstm_mae = mean_absolute_error(lstm_targets, lstm_preds)
    lstm_rmse = np.sqrt(mean_squared_error(lstm_targets, lstm_preds))

    mae_list.append(lstm_mae)
    rmse_list.append(lstm_rmse)

    print(f"Run {run+1} - LSTM MAE: {lstm_mae:.4f} | RMSE: {lstm_rmse:.4f}")

# Compute mean metrics
mean_lstm_mae = np.mean(mae_list)
mean_lstm_rmse = np.mean(rmse_list)

print("\n======= FINAL RESULTS =======")
print(f"Mean LSTM MAE  : {mean_lstm_mae:.4f}")
print(f"Mean LSTM RMSE : {mean_lstm_rmse:.4f}")


Train size: 1000, Test size: 1000

=== Run 1/10 ===
LSTM Epoch 1/100, Train Loss: 0.0317
LSTM Epoch 10/100, Train Loss: 0.0045
LSTM Epoch 20/100, Train Loss: 0.0041
LSTM Epoch 30/100, Train Loss: 0.0039
LSTM Epoch 40/100, Train Loss: 0.0038
LSTM Epoch 50/100, Train Loss: 0.0040
LSTM Epoch 60/100, Train Loss: 0.0038
LSTM Epoch 70/100, Train Loss: 0.0037
LSTM Epoch 80/100, Train Loss: 0.0036
LSTM Epoch 90/100, Train Loss: 0.0037
LSTM Epoch 100/100, Train Loss: 0.0036
Run 1 - LSTM MAE: 0.0459 | RMSE: 0.0651

=== Run 2/10 ===
LSTM Epoch 1/100, Train Loss: 0.0500
LSTM Epoch 10/100, Train Loss: 0.0052
LSTM Epoch 20/100, Train Loss: 0.0041
LSTM Epoch 30/100, Train Loss: 0.0041
LSTM Epoch 40/100, Train Loss: 0.0039
LSTM Epoch 50/100, Train Loss: 0.0038
LSTM Epoch 60/100, Train Loss: 0.0038
LSTM Epoch 70/100, Train Loss: 0.0036
LSTM Epoch 80/100, Train Loss: 0.0037
LSTM Epoch 90/100, Train Loss: 0.0037
LSTM Epoch 100/100, Train Loss: 0.0036
Run 2 - LSTM MAE: 0.0462 | RMSE: 0.0656

=== Run 3/10 

## AR(p)

In [13]:
# Load and preprocess data
df = pd.read_csv('data/Sunspots_clean.csv')  
values = df['Sunspots'].values

# Normalize data to [0, 1]
scaler = MinMaxScaler()
scaled_series = scaler.fit_transform(values.reshape(-1, 1)).flatten()

# Split into training and test sets
train_set = scaled_series[:1000]
test_set = scaled_series[1000:2000]

print (f"Train size: {len(train_set)}, Test size: {len(test_set)}")

# Prepare AR(p) design matrix for training
p = 6  # AR order
T_train = len(train_set)

# Create lagged input matrix (shape: [T - p, p])
X_train = np.column_stack([train_set[i:T_train - p + i] for i in range(p)])
y_train = train_set[p:]

# Solve OLS: phi = (X^T X)^{-1} X^T y
phi_hat = np.linalg.inv(X_train.T @ X_train) @ (X_train.T @ y_train)

print(f"Estimated phi coefficients (AR({p})): {phi_hat.round(4)}")

# 4. Predict on test set
# Start with last p values of training set
prev_vals = list(train_set[-p:])
preds = []

for t in range(len(test_set)):
    x_input = np.array(prev_vals[-p:])  # most recent 6 values
    x_hat = np.dot(phi_hat, x_input)
    preds.append(x_hat)
    prev_vals.append(test_set[t])  # update with true value (for one-step-ahead)

# 5. Compute MAE and RMSE
ar_mae = np.mean(np.abs(test_set - preds))
ar_rmse = np.sqrt(mean_squared_error(test_set, preds))
print(f"Test MAE (AR(p)): {ar_mae:.4f}")
print(f"Test RMSE (AR(p)): {ar_rmse:.4f}")

Train size: 1000, Test size: 1000
Estimated phi coefficients (AR(6)): [0.0361 0.055  0.1245 0.1148 0.1265 0.526 ]
Test MAE (AR(p)): 0.0465
Test RMSE (AR(p)): 0.0652


In [19]:
# Compare model 
print(f"RIB-GRU MAE: {mean_rib_gru_mae:.4f} | RMSE: {mean_rib_gru_rmse:.4f}")
print(f"iproj MAE  : {mean_iproj_mae:.4f} | RMSE: {mean_iproj_rmse:.4f}")
print(f"LSTM MAE   : {mean_lstm_mae:.4f} | RMSE: {mean_lstm_rmse:.4f}")
print(f"AR(p) MAE  : {ar_mae:.4f} | RMSE: {ar_rmse:.4f}")

RIB-GRU MAE: 0.0469 | RMSE: 0.0655
iproj MAE  : 0.0470 | RMSE: 0.0657
LSTM MAE   : 0.0464 | RMSE: 0.0649
AR(p) MAE  : 0.0465 | RMSE: 0.0652


# Data Type II

## IB-GRU

In [33]:
# Load data
df = pd.read_csv('RIB Paper/data/Sunspots.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.rename(columns={'Monthly Mean Total Sunspot Number': 'Sunspots'}, inplace=True)
values = df['Sunspots'].values

# Normalize data to [0, 1]
scaler = MinMaxScaler()
scaled_series = scaler.fit_transform(values.reshape(-1, 1)).flatten()

# Split into training and test sets
n = len(scaled_series) * 0.8
train_set = scaled_series[:int(n)]
test_set = scaled_series[int(n):]

print (f"Train size: {len(train_set)}, Test size: {len(test_set)}")

# Ensure the sequences are of the same length
X_train = torch.tensor(train_set[:-1]).unsqueeze(0).float()  
Y_train = torch.tensor(train_set[1:]).unsqueeze(0).float()   

X_val = torch.tensor(test_set[:-1]).unsqueeze(0).float()
Y_val = torch.tensor(test_set[1:]).unsqueeze(0).float()

# Create DataLoader for training and validation sets
train_loader = DataLoader(TensorDataset(X_train, Y_train), batch_size=1, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, Y_val), batch_size=1, shuffle=False)

# Initialize model
rib_gru_model = RIB_GRU_Model(input_dim=1, hidden_dim=32, z_dim=16)
optimizer = optim.Adam(rib_gru_model.parameters(), lr=0.001)
beta = 0.005
num_epochs = 1000

for epoch in range(num_epochs):
    rib_gru_model.train()
    total_loss = 0.0

    for X_batch, Y_batch in train_loader:
        h_t = torch.zeros(X_batch.size(0), rib_gru_model.hidden_dim)

        loss = 0.0
        for t in range(X_batch.size(1)):
            x_t = X_batch[:, t].unsqueeze(-1)
            y_t = Y_batch[:, t].unsqueeze(-1)

            out = rib_gru_model(x_t, h_t)
            h_t = out['h_t']

            # Compute loss
            loss += rib_loss(out, y_t, beta=beta)
            
        # Average loss over the batch
        loss /= X_batch.size(1)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    if (epoch + 1) % 100 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_loss:.4f}")

#13mins

Train size: 2612, Test size: 653
Epoch 1/1000 | Train Loss: 0.9967
Epoch 100/1000 | Train Loss: -0.3437
Epoch 200/1000 | Train Loss: -0.4719
Epoch 300/1000 | Train Loss: -1.1600
Epoch 400/1000 | Train Loss: -1.2661
Epoch 500/1000 | Train Loss: -1.3551
Epoch 600/1000 | Train Loss: -1.4462
Epoch 700/1000 | Train Loss: -1.4515
Epoch 800/1000 | Train Loss: -1.4584
Epoch 900/1000 | Train Loss: -1.4906
Epoch 1000/1000 | Train Loss: -1.5108


In [60]:
# Store metrics from all runs
mae_list = []
rmse_list = []

for run in range(10):

    # Validation
    rib_gru_model.eval()
    preds = []
    trues = []
    vars_all = []

    with torch.no_grad():
        for X_batch, Y_batch in val_loader:

            h_t = torch.zeros(X_batch.size(0), rib_gru_model.hidden_dim)
            pred_mu = []
            pred_var = []

            for t in range(X_batch.size(1)):

                x_t = X_batch[:, t].unsqueeze(-1)
                y_t = Y_batch[:, t].unsqueeze(-1)

                out = rib_gru_model(x_t, h_t)
                h_t = out['h_t']

                mu_dec = out['mu_dec']
                logvar_dec = out['logvar_dec']

                pred_mu.append(mu_dec.squeeze(-1))
                pred_var.append(torch.exp(logvar_dec).squeeze(-1))

            pred_mu = torch.stack(pred_mu, dim=1)  # [1, T]
            pred_var = torch.stack(pred_var, dim=1)  # [1, T]

            preds.append(pred_mu)
            vars_all.append(pred_var)
            trues.append(Y_batch)

    preds = torch.cat(preds, dim=0).squeeze(0).numpy()
    trues = torch.cat(trues, dim=0).squeeze(0).numpy()
    vars_all = torch.cat(vars_all, dim=0).squeeze(0).numpy()

    # Calculate MAE, RMSE
    rib_gru_mae = np.mean(np.abs(preds - trues))
    rib_gru_rmse = np.sqrt(np.mean((preds - trues)**2))

    mae_list.append(rib_gru_mae)
    rmse_list.append(rib_gru_rmse)

    print(f"Run {run+1}: RIB-GRU MAE: {rib_gru_mae:.4f} | RMSE: {rib_gru_rmse:.4f}")

# Compute mean metrics
mean_rib_gru_mae = np.mean(mae_list)
mean_rib_gru_rmse = np.mean(rmse_list)

print("======= FINAL RESULTS =======")
print(f"Mean RIB-GRU MAE: {mean_rib_gru_mae:.4f} | RMSE: {mean_rib_gru_rmse:.4f}")

Run 1: RIB-GRU MAE: 0.0424 | RMSE: 0.0566
Run 2: RIB-GRU MAE: 0.0427 | RMSE: 0.0573
Run 3: RIB-GRU MAE: 0.0424 | RMSE: 0.0561
Run 4: RIB-GRU MAE: 0.0425 | RMSE: 0.0568
Run 5: RIB-GRU MAE: 0.0426 | RMSE: 0.0568
Run 6: RIB-GRU MAE: 0.0423 | RMSE: 0.0568
Run 7: RIB-GRU MAE: 0.0420 | RMSE: 0.0562
Run 8: RIB-GRU MAE: 0.0422 | RMSE: 0.0566
Run 9: RIB-GRU MAE: 0.0425 | RMSE: 0.0567
Run 10: RIB-GRU MAE: 0.0422 | RMSE: 0.0566
======= FINAL RESULTS =======
Mean RIB-GRU MAE: 0.0424 | RMSE: 0.0567


## IProjLSTM

In [26]:
# Load data
df = pd.read_csv('RIB Paper/data/Sunspots.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.rename(columns={'Monthly Mean Total Sunspot Number': 'Sunspots'}, inplace=True)
values = df['Sunspots'].values

# Normalize data to [0, 1]
scaler = MinMaxScaler()
scaled_series = scaler.fit_transform(values.reshape(-1, 1)).flatten()

# Split into training and test sets
n = len(scaled_series) * 0.8
train_set = scaled_series[:int(n)]
test_set = scaled_series[int(n):]

print (f"Train size: {len(train_set)}, Test size: {len(test_set)}")

# Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len=20): 
        self.series = torch.tensor(series, dtype=torch.float32)
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        x = self.series[idx:idx+self.seq_len]
        y = self.series[idx+self.seq_len]
        return x.unsqueeze(-1), y.unsqueeze(-1)

# Parameters
seq_len = 16
batch_size = 32

# DataLoaders
train_dataset = TimeSeriesDataset(train_set, seq_len)
test_dataset = TimeSeriesDataset(test_set, seq_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Train size: 2612, Test size: 653


In [27]:
# Lists to store metrics for all runs
mae_list = []
rmse_list = []

for run in range(10):
    print(f"\n=== Run {run+1}/10 ===")

    # Model setup
    iproj_model = IProjLSTMModel(input_dim=1, hidden_dim=32, seq_len=seq_len)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    iproj_model = iproj_model.to(device)

    optimizer = torch.optim.Adam(iproj_model.parameters(), lr=0.001)
    criterion = torch.nn.MSELoss()

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        iproj_model.train()
        epoch_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = iproj_model(x_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}")

    # Evaluation
    iproj_model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(device)
            y_pred = iproj_model(x_batch)
            preds.append(y_pred.cpu())
            targets.append(y_batch.cpu())

    preds = torch.cat(preds).squeeze().numpy()
    targets = torch.cat(targets).squeeze().numpy()

    iproj_mae = mean_absolute_error(targets, preds)
    iproj_rmse = np.sqrt(mean_squared_error(targets, preds))

    mae_list.append(iproj_mae)
    rmse_list.append(iproj_rmse)

    print(f"Run {run+1} - Test MAE: {iproj_mae:.4f} | Test RMSE: {iproj_rmse:.4f}")

# Compute mean metrics
mean_iproj_mae = np.mean(mae_list)
mean_iproj_rmse = np.mean(rmse_list)

print("\n======= FINAL RESULTS =======")
print(f"Mean MAE  : {mean_iproj_mae:.4f}")
print(f"Mean RMSE : {mean_iproj_rmse:.4f}")



=== Run 1/10 ===
Epoch 1/100, Train Loss: 0.0128
Epoch 10/100, Train Loss: 0.0041
Epoch 20/100, Train Loss: 0.0041
Epoch 30/100, Train Loss: 0.0041
Epoch 40/100, Train Loss: 0.0041
Epoch 50/100, Train Loss: 0.0042
Epoch 60/100, Train Loss: 0.0041
Epoch 70/100, Train Loss: 0.0040
Epoch 80/100, Train Loss: 0.0040
Epoch 90/100, Train Loss: 0.0039
Epoch 100/100, Train Loss: 0.0039
Run 1 - Test MAE: 0.0403 | Test RMSE: 0.0563

=== Run 2/10 ===
Epoch 1/100, Train Loss: 0.0081
Epoch 10/100, Train Loss: 0.0041
Epoch 20/100, Train Loss: 0.0041
Epoch 30/100, Train Loss: 0.0042
Epoch 40/100, Train Loss: 0.0041
Epoch 50/100, Train Loss: 0.0041
Epoch 60/100, Train Loss: 0.0042
Epoch 70/100, Train Loss: 0.0040
Epoch 80/100, Train Loss: 0.0039
Epoch 90/100, Train Loss: 0.0040
Epoch 100/100, Train Loss: 0.0041
Run 2 - Test MAE: 0.0423 | Test RMSE: 0.0565

=== Run 3/10 ===
Epoch 1/100, Train Loss: 0.0433
Epoch 10/100, Train Loss: 0.0042
Epoch 20/100, Train Loss: 0.0042
Epoch 30/100, Train Loss: 0.0044

## LSTM

In [28]:
# Load data
df = pd.read_csv('RIB Paper/data/Sunspots.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.rename(columns={'Monthly Mean Total Sunspot Number': 'Sunspots'}, inplace=True)
values = df['Sunspots'].values

# Normalize data to [0, 1]
scaler = MinMaxScaler()
scaled_series = scaler.fit_transform(values.reshape(-1, 1)).flatten()

# Split into training and test sets
n = len(scaled_series) * 0.8
train_set = scaled_series[:int(n)]
test_set = scaled_series[int(n):]

print (f"Train size: {len(train_set)}, Test size: {len(test_set)}")

# Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len=20): 
        self.series = torch.tensor(series, dtype=torch.float32)
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        x = self.series[idx:idx+self.seq_len]
        y = self.series[idx+self.seq_len]
        return x.unsqueeze(-1), y.unsqueeze(-1)

# Parameters
seq_len = 16
batch_size = 32

# DataLoaders
train_dataset = TimeSeriesDataset(train_set, seq_len)
test_dataset = TimeSeriesDataset(test_set, seq_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Train size: 2612, Test size: 653


In [31]:
# Lists to store metrics for all runs
mae_list = []
rmse_list = []

for run in range(10):
    print(f"\n=== Run {run+1}/10 ===")

    # Initialize model
    lstm_model = LSTMModel(input_dim=1, hidden_dim=32, seq_len=seq_len)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    lstm_model = lstm_model.to(device)

    optimizer_lstm = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    num_epochs = 100
    # Training loop
    for epoch in range(num_epochs):
        lstm_model.train()
        epoch_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer_lstm.zero_grad()
            y_pred = lstm_model(x_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer_lstm.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)

        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"LSTM Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}")

    # Evaluation
    lstm_model.eval()
    lstm_preds = []
    lstm_targets = []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(device)
            y_pred = lstm_model(x_batch)
            lstm_preds.append(y_pred.cpu())
            lstm_targets.append(y_batch.cpu())

    lstm_preds = torch.cat(lstm_preds).squeeze().numpy()
    lstm_targets = torch.cat(lstm_targets).squeeze().numpy()

    lstm_mae = mean_absolute_error(lstm_targets, lstm_preds)
    lstm_rmse = np.sqrt(mean_squared_error(lstm_targets, lstm_preds))

    mae_list.append(lstm_mae)
    rmse_list.append(lstm_rmse)

    print(f"Run {run+1} - LSTM MAE: {lstm_mae:.4f} | RMSE: {lstm_rmse:.4f}")

# Compute mean metrics
mean_lstm_mae = np.mean(mae_list)
mean_lstm_rmse = np.mean(rmse_list)

print("\n======= FINAL RESULTS =======")
print(f"Mean LSTM MAE  : {mean_lstm_mae:.4f}")
print(f"Mean LSTM RMSE : {mean_lstm_rmse:.4f}")



=== Run 1/10 ===
LSTM Epoch 1/100, Train Loss: 0.0450
LSTM Epoch 10/100, Train Loss: 0.0046
LSTM Epoch 20/100, Train Loss: 0.0042
LSTM Epoch 30/100, Train Loss: 0.0042
LSTM Epoch 40/100, Train Loss: 0.0040
LSTM Epoch 50/100, Train Loss: 0.0040
LSTM Epoch 60/100, Train Loss: 0.0040
LSTM Epoch 70/100, Train Loss: 0.0041
LSTM Epoch 80/100, Train Loss: 0.0040
LSTM Epoch 90/100, Train Loss: 0.0040
LSTM Epoch 100/100, Train Loss: 0.0039
Run 1 - LSTM MAE: 0.0408 | RMSE: 0.0562

=== Run 2/10 ===
LSTM Epoch 1/100, Train Loss: 0.0220
LSTM Epoch 10/100, Train Loss: 0.0044
LSTM Epoch 20/100, Train Loss: 0.0043
LSTM Epoch 30/100, Train Loss: 0.0041
LSTM Epoch 40/100, Train Loss: 0.0040
LSTM Epoch 50/100, Train Loss: 0.0040
LSTM Epoch 60/100, Train Loss: 0.0040
LSTM Epoch 70/100, Train Loss: 0.0041
LSTM Epoch 80/100, Train Loss: 0.0040
LSTM Epoch 90/100, Train Loss: 0.0040
LSTM Epoch 100/100, Train Loss: 0.0039
Run 2 - LSTM MAE: 0.0405 | RMSE: 0.0559

=== Run 3/10 ===
LSTM Epoch 1/100, Train Loss: 

## AR(p)

In [ ]:
# Load data
df = pd.read_csv('RIB Paper/data/Sunspots.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.rename(columns={'Monthly Mean Total Sunspot Number': 'Sunspots'}, inplace=True)
values = df['Sunspots'].values

# Normalize data to [0, 1]
scaler = MinMaxScaler()
scaled_series = scaler.fit_transform(values.reshape(-1, 1)).flatten()

# Split into training and test sets
n = len(scaled_series) * 0.8
train_set = scaled_series[:int(n)]
test_set = scaled_series[int(n):]

print (f"Train size: {len(train_set)}, Test size: {len(test_set)}")

# Prepare AR(6) design matrix for training
p = 5  # AR order
T_train = len(train_set)

# Create lagged input matrix (shape: [T - p, p])
X_train = np.column_stack([train_set[i:T_train - p + i] for i in range(p)])
y_train = train_set[p:]

# Solve OLS: phi = (X^T X)^{-1} X^T y
phi_hat = np.linalg.inv(X_train.T @ X_train) @ (X_train.T @ y_train)

print(f"Estimated phi coefficients (AR(p)): {phi_hat.round(4)}")

# 4. Predict on test set
# Start with last p values of training set
prev_vals = list(train_set[-p:])
preds = []

for t in range(len(test_set)):
    x_input = np.array(prev_vals[-p:])  # most recent 6 values
    x_hat = np.dot(phi_hat, x_input)
    preds.append(x_hat)
    prev_vals.append(test_set[t])  # update with true value (for one-step-ahead)

# 5. Compute MAE and RMSE
ar_mae = np.mean(np.abs(test_set - preds))
ar_rmse = np.sqrt(mean_squared_error(test_set, preds))
print(f"Test MAE (AR(p)): {ar_mae:.4f}")
print(f"Test RMSE (AR(6\p)): {ar_rmse:.4f}")

Train size: 2612, Test size: 653
Estimated phi coefficients (AR(6)): [0.0759 0.095  0.1028 0.1245 0.5855]
Test MAE (AR(6)): 0.0419
Test RMSE (AR(6)): 0.0579


In [63]:
# Compare results of these 3 model
print(f"RIB-GRU MAE:   {mean_rib_gru_mae:.4f} | RMSE: {mean_rib_gru_rmse:.4f}")
print(f"IProjLSTM MAE: {mean_iproj_mae:.4f} | RMSE: {mean_iproj_rmse:.4f}")
print(f"LSTM MAE:      {mean_lstm_mae:.4f} | RMSE: {mean_lstm_rmse:.4f}")
print(f"AR(p) MAE:     {ar_mae:.4f} | RMSE: {ar_rmse:.4f}")

RIB-GRU MAE:   0.0424 | RMSE: 0.0567
IProjLSTM MAE: 0.0412 | RMSE: 0.0564
LSTM MAE:      0.0415 | RMSE: 0.0566
AR(p) MAE:     0.0419 | RMSE: 0.0579
